In [2]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import RandomForestRegressor

from sklearn import cross_validation

import numpy
import math

In [3]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Store,Date,,,,,,,
1,2015-07-31,5,5263,555,1,1,0,1
2,2015-07-31,5,6064,625,1,1,0,1
3,2015-07-31,5,8314,821,1,1,0,1
4,2015-07-31,5,13995,1498,1,1,0,1
5,2015-07-31,5,4822,559,1,1,0,1


In [4]:
# Featurize
train_df.reset_index(inplace=True)
train_df['Date'] = train_df['Date'].map(lambda x: x.value)
def numericStateHoliday(x):
    if x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    else:
        return x
train_df['StateHoliday'] = train_df['StateHoliday'].map(lambda x: numericStateHoliday(x))

In [5]:
def train_and_score(training_df, features):

    regressor = RandomForestRegressor(n_estimators=25, n_jobs=-1)
    predict = cross_validation.cross_val_predict(regressor, training_df[features], training_df['Sales'], n_jobs=-1)
    
    score_df = training_df[['Sales']].copy()
    score_df['Predict'] = predict
    score_df = score_df[score_df['Sales'] != 0]
    score_df['PercentError'] =  (score_df['Sales'] - score_df['Predict']) / score_df['Sales']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [6]:
%%time

features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
print("Score: {}".format(train_and_score(train_df, features)))

Score: 0.40479154499495
CPU times: user 708 ms, sys: 668 ms, total: 1.38 s
Wall time: 29.4 s


In [7]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Store,Date,,,,,,,
1,2015-07-31,5,5263,555,1,1,0,1
2,2015-07-31,5,6064,625,1,1,0,1
3,2015-07-31,5,8314,821,1,1,0,1
4,2015-07-31,5,13995,1498,1,1,0,1
5,2015-07-31,5,4822,559,1,1,0,1


In [8]:
# Featurize v2
train_df.reset_index(inplace=True)
def numericStateHoliday(x):
    if x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    else:
        return x
train_df['StateHoliday'] = train_df['StateHoliday'].map(lambda x: numericStateHoliday(x))

In [10]:

def featurizeDate(ts): 
    features = {}
    features['cal:dayOfMonth'] = ts.day
    features['cal:dayOfWeek'] = ts.dayofweek
    features['cal:dayofyear'] = ts.dayofyear
    features['cal:month'] = ts.month
    features['cal:year'] = ts.year
    features['cal:weekofyear'] =  ts.weekofyear
    features['cal:quarter'] = ts.quarter

    return features



date_features_series = train_df['Date'].map(lambda x: featurizeDate(x))
date_features = DataFrame.from_records(date_features_series.tolist(), index=train_df['Date'])


In [13]:
len(date_features)

1017209

In [22]:
date_features.index

DatetimeIndex(['2015-07-31', '2015-07-31', '2015-07-31', '2015-07-31',
               '2015-07-31', '2015-07-31', '2015-07-31', '2015-07-31',
               '2015-07-31', '2015-07-31', 
               ...
               '2013-01-01', '2013-01-01', '2013-01-01', '2013-01-01',
               '2013-01-01', '2013-01-01', '2013-01-01', '2013-01-01',
               '2013-01-01', '2013-01-01'],
              dtype='datetime64[ns]', name='Date', length=1017209, freq=None, tz=None)

In [24]:
train_df = pandas.concat([train_df, date_features.reset_index()], axis=1)


In [25]:
del train_df['Date']
# We just use the DayOfWeek param provided by Kaggle. One is 0-based, the other is 1-based, but no matter.
del train_df['cal:dayOfWeek']

In [26]:
%%time

features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
print("Score: {}".format(train_and_score(train_df, features)))

Score: 0.3381351366997324
CPU times: user 1.5 s, sys: 1.19 s, total: 2.69 s
Wall time: 48.3 s
